In [1]:
import statsmodels.api as sm
import numpy as np

def get_rank_demeaned_normalized_signal(raw_signal):
    signal_rank = raw_signal.rank(axis=1)
    signal_mean = raw_signal.rank(axis=1).mean(axis=1)
    demeaned_signal = signal_rank.subtract(signal_mean, axis=0)
    return demeaned_signal.divide(demeaned_signal.abs().sum(axis=1), axis=0)


def get_gross_returns_and_net_returns(signal_weights, px):
    asset_returns = px / px.shift() - 1
    weighted_returns = signal_weights.shift() * asset_returns
    gross_returns = weighted_returns.sum(axis=1)
    turnover = (signal_weights.fillna(0) - signal_weights.shift().fillna(0)).abs().sum(axis=1)
    tcost_bps = 20 # (commission + slippage)
    net_returns = gross_returns.subtract(turnover * tcost_bps * 1e-4, fill_value = 0)
    return gross_returns, net_returns


# returns pair in the form of (alpha, beta)
def get_alpha_beta_to_asset(net_returns, benchmark_asset_returns):
    model = sm.OLS(net_returns, sm.add_constant(benchmark_asset_returns))
    res = model.fit()
    return res.params[0], res.params[1]


def get_max_drawdown(net_returns):
    cumulative_net_returns = net_returns.cumsum()
    drawdowns = cumulative_net_returns / cumulative_net_returns.expanding(min_periods=1).max() - 1
    return drawdowns[drawdowns != float('-inf')].min()


def get_max_drawdown_duration(net_returns, hours_freq):
    cumulative_net_returns = net_returns.cumsum()
    
    peak = cumulative_net_returns.expanding(min_periods=1).max()
    
    max_drawdown_duration = 0
    current_drawdown_duration = 0
    
    for dt in cumulative_net_returns.index:
        if cumulative_net_returns[dt] >= peak[dt]:
            current_drawdown_duration = 0
        else:
            current_drawdown_duration += 1
            max_drawdown_duration = max(max_drawdown_duration, current_drawdown_duration)
    return max_drawdown_duration * hours_freq / 24


# trade_hours_freq = 4, 8, 12, 24 (for 1 day), ...
def get_strategy_stats(net_returns, trade_hours_freq, input_prices):
    
    bitcoin_returns_over_period = input_prices['BTCUSDT'] / input_prices['BTCUSDT'].shift() - 1
    
    alpha, beta = get_alpha_beta_to_asset(net_returns.iloc[2:],
                                          bitcoin_returns_over_period.iloc[2:])
    res = {
        "avg returns": net_returns.mean() * 24 / trade_hours_freq * 365,
        "volatility": net_returns.std() * np.sqrt(24 / trade_hours_freq * 365),
        "sharpe ratio": net_returns.mean() / net_returns.std() * np.sqrt(24 / trade_hours_freq * 365),
        "max drawdown": get_max_drawdown(net_returns),
        "max drawdown duration": get_max_drawdown_duration(net_returns, trade_hours_freq),
        "alpha_BTC": alpha,
        "beta_BTC": beta,
    }
    return res

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
